#Problème - Session n°2 : une variable cachée

Dans ce problème, on travaille sur un jeu de données comportant 50.000 entrées $x_i$ et des cibles $y_i$. Les entrées sont des vecteurs de taille 10 (au format torch), les cibles sont des scalaires construits à partir de cinq fonctions différentes ($f_0$, ..., $f_4$) : \

$$ \forall i, \exists k\in [\![0 \;;4]\!]  \:\: \text{tel que} \: f_k(x_i) = y_i $$

Ces fonctions sont inconnues, ainsi que l'indice $k$. Par contre, on sait que le groupe des 1000 premières cibles ont été construites à partir du même indice  $k$, de même pour les mille  suivantes, et ainsi de suite.

Le but est de parvenir à rassembler les groupes de cibles qui ont été générées avec le même indice $k$ (avec la même fonction).

In [4]:
# Example d'échantillonnage du dataset
import torch
from torch.utils.data import DataLoader

! git clone https://github.com/Zakaria-Yahya/exam_2025_session2.git
! cp exam_2025_session2/utils/utils.py .
from utils import Problem1Dataset

dataset = Problem1Dataset()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in dataloader:
    x_batch, y_batch, k_batch, idx_batch = batch
    print("Batch input shape:", x_batch.shape)
    print("Batch target shape:", y_batch.shape)
    print("Batch k shape:", k_batch.shape) # indice k (pas utilisable à l'entraînement)
    print("Batch indices shape:", idx_batch.shape)
    break

Cloning into 'exam_2025_session2'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 93 (delta 28), reused 8 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 729.62 KiB | 2.35 MiB/s, done.
Resolving deltas: 100% (28/28), done.
Batch input shape: torch.Size([32, 10])
Batch target shape: torch.Size([32, 1])
Batch k shape: torch.Size([32])
Batch indices shape: torch.Size([32])


**Consignes :**
- Entraîner l'architecture proposée dans la cellule suivante.
- Montrer que les vecteurs 2D de self.theta permettent de répondre
  au problème posé.
- Décrire le rôle de self.theta, du vector noise \
 et ainsi que la raison de la division par 1000 (**indices // 1000** dans le code).

In [7]:
import torch.nn as nn
class DeepMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super(DeepMLP, self).__init__()
        self.theta = nn.Parameter(torch.randn(50, 2))
        self.fc1 = nn.Linear(input_dim + 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, indices):
        theta_batch = self.theta[indices // 1000, :]
        noise = torch.normal(mean=torch.zeros_like(theta_batch),
                             std=torch.ones_like(theta_batch))
        x = torch.cat([x, theta_batch + noise], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x, theta_batch

In [11]:

from utils import Problem1Dataset



# Chargement du jeu de données
jeu_donnees = Problem1Dataset()
chargeur_donnees = DataLoader(jeu_donnees, batch_size=128, shuffle=True)  # Augmentation de la taille du lot pour un entraînement plus rapide

# Vérification de la disponibilité du GPU et définition du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device utilisé : {device}")

# Initialisation du modèle
dimension_entree = 10  # Chaque entrée est un vecteur de taille 10
dimension_sortie = 1   # Chaque cible est un scalaire
modele = DeepMLP(dimension_entree, dimension_sortie).to(device)  # Déplacement du modèle vers le GPU

# Définition de la fonction de perte et de l'optimiseur
fonction_perte = nn.MSELoss()  # Erreur quadratique moyenne pour la régression
optimiseur = optim.Adam(modele.parameters(), lr=0.001)  # Optimiseur Adam

# Boucle d'entraînement
nombre_epochs = 500
for epoch in range(nombre_epochs):
    for lot in chargeur_donnees:
        x_lot, y_lot, _, indices_lot = lot

        # Déplacement des données vers le GPU
        x_lot = x_lot.to(device)
        y_lot = y_lot.to(device)
        indices_lot = indices_lot.to(device)

        # Remise à zéro des gradients
        optimiseur.zero_grad()

        # Passe avant
        predictions, theta_lot = modele(x_lot, indices_lot)

        # Calcul de la perte
        perte = fonction_perte(predictions, y_lot)

        # Passe arrière et optimisation
        perte.backward()
        optimiseur.step()

    # Affichage de la perte toutes les 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch [{epoch+1}/{nombre_epochs}], Perte : {perte.item():.4f}")

# Après l'entraînement, inspection des vecteurs theta appris
print("Vecteurs theta appris :")
print(modele.theta)

device utilisé : cuda
Epoch [1/500], Perte : 210.4705
Epoch [101/500], Perte : 1.0804
Epoch [201/500], Perte : 0.5809
Epoch [301/500], Perte : 0.2401
Epoch [401/500], Perte : 0.1871
Vecteurs theta appris :
Parameter containing:
tensor([[ 5.3656,  8.0830],
        [ 0.0845, -1.7936],
        [-7.8884,  5.8267],
        [ 7.7862, -8.4657],
        [-9.0142, -7.3800],
        [ 5.7689,  7.6464],
        [-0.0484, -1.6307],
        [-7.4704,  5.9410],
        [ 7.9559, -7.4387],
        [-9.2191, -7.8458],
        [ 5.7627,  7.9818],
        [-0.0897, -1.6833],
        [-7.6012,  5.9153],
        [ 7.9696, -8.1819],
        [-9.1198, -7.5318],
        [ 5.5893,  7.6195],
        [-0.0953, -1.6432],
        [-7.6828,  5.7971],
        [ 8.0401, -7.9132],
        [-8.8164, -8.3767],
        [ 5.4339,  8.0350],
        [ 0.1147, -1.4896],
        [-7.6254,  5.8859],
        [ 8.2079, -7.1375],
        [-8.1892, -7.8631],
        [ 5.5997,  7.8303],
        [-0.0811, -1.6699],
        [-7.3679

- 2) L'objectif est de regrouper les données qui ont été générées avec la même fonction $f_k$$f_k$. Pour ce faire, le modèle apprend un vecteur 2D unique ( self.theta[k] ) pour chaque fonction $f_k$ (k allant de 0 à 4).Durant l'entraînement, self.theta est ajusté de telle sorte que les points appartenant au même groupe (même fonction $f_k$) soient proches dans l'espace 2D défini par self.theta.Tu peux vérifier cela après l'entraînement en visualisant les 50 vecteurs 2D de self.theta (qui correspondent aux 50 groupes de 1000 points) : les vecteurs correspondant au même k devraient être regroupés.

- 3)**"self.theta" :** C'est le cœur du système de regroupement. Il stocke 50 vecteurs 2D, un pour chaque groupe de 1000 points. Chaque vecteur représente un point dans un espace latent 2D. Le modèle apprend à positionner ces vecteurs de manière à ce que les points appartenant au même groupe (même fonction $f_k$$f_k$) aient des vecteurs theta proches. **"noise" :** Le bruit gaussien ajouté à theta_batch agit comme une régularisation. Il empêche le modèle de simplement mémoriser les positions de theta pour chaque point et le force à apprendre une représentation plus générale. Cela permet au modèle de mieux généraliser à de nouvelles données. **"indices // 1000"** : Cette opération permet de récupérer l'indice du groupe (0 à 49) auquel appartient un point donné. Le jeu de données est construit de telle sorte que les 1000 premières données appartiennent au groupe 0, les 1000 suivantes au groupe 1, etc. En divisant l'indice global d'une donnée par 1000, on obtient l'indice du groupe auquel elle appartient. Ce groupe est utilisé ensuite pour récupérer le vecteur theta correspondant.